In [1]:
import new_pipe
import has_violation

import seaborn as sns
from aequitas.group import Group
from aequitas.bias import Bias
from aequitas.fairness import Fairness
from aequitas.plotting import Plot

In [2]:
df = has_violation.go()  # this is the features dataframe

/Users/rachelwhaley/Documents/MPCS/30254-MLPP/project/epa-rcra-violations/full_pipeline_files/has_violation.py:70: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  prev_vios = pd.concat([prev_vios, vios_this_yr], ignore_index=True)


In [3]:
df.shape

(8330032, 43)

In [4]:
features_df = new_pipe.add_acs_features(df)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8330032 entries, 0 to 8330031
Data columns (total 44 columns):
ID_NUMBER                              object
YEAR_EVALUATED                         int64
HasViolation                           float64
DAYS_SINCE_VIOLATIONS_IN_ID_NUMBER     float64
DAYS_SINCE_VIOLATIONS_IN_STATE_CODE    float64
DAYS_SINCE_VIOLATIONS_IN_ZIP_CODE      float64
NUMBER_VIOLATIONS_IN_ID_NUMBER         float64
NUMBER_VIOLATIONS_IN_STATE_CODE        float64
NUMBER_VIOLATIONS_IN_ZIP_CODE          float64
IsLQG                                  int64
IsTSDF                                 int64
NumInMyState                           float64
NumInMyZIP                             float64
early  ID_NUMBER avg                   float64
early  ID_NUMBER sum                   float64
early  ID_NUMBER count                 float64
last early  ID_NUMBER                  float64
early  ZIP_CODE avg                    float64
early  ZIP_CODE sum                    float64
e

In [7]:
import pandas as pd
import epa_pipeline as ep

In [9]:
def add_acs_features(df):
    acs = pd.read_csv('all_acs_data.csv')
    acs['median income'].fillna(0, inplace=True)
    acs['median income'] = acs['median income'].apply(lambda x: x if x > 0
                                                         else 0)
    df['acs_year'] = df['YEAR_EVALUATED'].where(df['YEAR_EVALUATED'] < 2017,
                                                   2016)
    df = df.merge(acs, left_on=['ID_NUMBER', 'acs_year'], right_on=['ID_NUMBER',
                                                                       'year'], how="left")
    df.drop('acs_year', axis=1, inplace=True)
    df.fillna(0, inplace=True)
    return df

In [10]:
acs_added_df = add_acs_features(df)

In [11]:
acs_added_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8330032 entries, 0 to 8330031
Data columns (total 70 columns):
ID_NUMBER                              object
YEAR_EVALUATED                         int64
HasViolation                           float64
DAYS_SINCE_VIOLATIONS_IN_ID_NUMBER     float64
DAYS_SINCE_VIOLATIONS_IN_STATE_CODE    float64
DAYS_SINCE_VIOLATIONS_IN_ZIP_CODE      float64
NUMBER_VIOLATIONS_IN_ID_NUMBER         float64
NUMBER_VIOLATIONS_IN_STATE_CODE        float64
NUMBER_VIOLATIONS_IN_ZIP_CODE          float64
IsLQG                                  int64
IsTSDF                                 int64
NumInMyState                           float64
NumInMyZIP                             float64
early  ID_NUMBER avg                   float64
early  ID_NUMBER sum                   float64
early  ID_NUMBER count                 float64
last early  ID_NUMBER                  float64
early  ZIP_CODE avg                    float64
early  ZIP_CODE sum                    float64
e

In [12]:
preds = pd.read_csv("predictions1.csv")

In [13]:
preds.shape

(1042512, 17)

In [17]:
preds_all = preds.merge(acs_added_df, left_index=True, right_index=True)

In [18]:
preds_all.shape

(1042512, 87)

In [19]:
preds_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1042512 entries, 0 to 1042511
Data columns (total 87 columns):
Unnamed: 0_x                                                                                                       1042512 non-null int64
DT{'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 1}_at_0.2pct                                          1042512 non-null int64
truth                                                                                                              1042512 non-null float64
DT{'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 5}_at_0.2pct                                          1042512 non-null int64
truth.1                                                                                                            1042512 non-null float64
DT{'min_samples_split': 2, 'criterion': 'gini', 'max_depth': None}_at_0.2pct                                       1042512 non-null int64
truth.2                                                

In [20]:
preds_all.fillna(0, inplace=True)

In [21]:
preds_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1042512 entries, 0 to 1042511
Data columns (total 87 columns):
Unnamed: 0_x                                                                                                       1042512 non-null int64
DT{'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 1}_at_0.2pct                                          1042512 non-null int64
truth                                                                                                              1042512 non-null float64
DT{'min_samples_split': 2, 'criterion': 'gini', 'max_depth': 5}_at_0.2pct                                          1042512 non-null int64
truth.1                                                                                                            1042512 non-null float64
DT{'min_samples_split': 2, 'criterion': 'gini', 'max_depth': None}_at_0.2pct                                       1042512 non-null int64
truth.2                                                

In [22]:
preds_all["families_under_poverty_line"] = preds_all["under_p5_x"] + preds_all["p5top74"] + preds_all["p75top99"] 

In [23]:
rf_for_aequitas = preds_all.filter(["RF{'min_samples_split': 10, 'max_depth': 5, 'n_estimators': 10, 'n_jobs': -1, 'max_features': 'sqrt'}_at_0.2pct", "truth.7", "median income", "black alone", "native alone", "p75top99"])

In [24]:
rf_for_aequitas.shape

(1042512, 6)

In [27]:
rf_for_aequitas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1042512 entries, 0 to 1042511
Data columns (total 6 columns):
RF{'min_samples_split': 10, 'max_depth': 5, 'n_estimators': 10, 'n_jobs': -1, 'max_features': 'sqrt'}_at_0.2pct    1042512 non-null int64
truth.7                                                                                                            1042512 non-null float64
median income                                                                                                      1042512 non-null float64
black alone                                                                                                        1042512 non-null float64
native alone                                                                                                       1042512 non-null float64
p75top99                                                                                                           1042512 non-null float64
dtypes: float64(5), int64(1)
memory usage: 55.7 MB

In [31]:
rf_for_aequitas = rf_for_aequitas.rename(columns={"RF{'min_samples_split': 10, 'max_depth': 5, 'n_estimators': 10, 'n_jobs': -1, 'max_features': 'sqrt'}_at_0.2pct": "score", "truth.7":"label_value"})


In [32]:
rf_for_aequitas["label_value"] = rf_for_aequitas["label_value"].astype(int)

In [33]:
g = Group()
xtab, _ = g.get_crosstabs(rf_for_aequitas)

Exception: get_crosstabs: input df was not preprocessed. There are non-string cols within attr_cols!